**INITIALIZE LIBRARY**

In [3]:
from ultralytics import YOLO
import cv2
from collections import deque, Counter
import re

**DOWNLOAD WEIGHT**

In [10]:
model = YOLO("./yolo12n.pt")

**TRAIN MODEL**

In [2]:
model.train(data="./dataset/data.yaml", epochs=100, imgsz=640)

NameError: name 'model' is not defined

In [11]:
model.val()

Ultralytics 8.3.90  Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLOv12n summary (fused): 159 layers, 2,590,824 parameters, 0 gradients, 6.5 GFLOPs


FileNotFoundError: Dataset 'None' for task=detect not found 

**YOLO INFERENCE NEW**

In [5]:
model_ocr = YOLO("./runs/detect/train/weights/bestOCR.pt")
model_plate = YOLO("./runs/detect/train/weights/bestLicensePlate.pt")

**REGEX**

In [6]:
PLATE_REGEX = re.compile(r"^([A-Z]{1,2})(\d{1,4})([A-Z]{2,3})$")

In [7]:
def normalize_plate(text):
    text = text.replace(" ", "").upper()
    match = PLATE_REGEX.match(text)
    if match:
        return f"{match.group(1)} {match.group(2)} {match.group(3)}"
    return None

In [8]:
plate_history = deque(maxlen=15)

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model_plate(frame)
    plate_img = None
    x1, y1 = 0, 0

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            plate_img = frame[y1:y2, x1:x2].copy()

            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    if plate_img is not None:
        ocr_results = model_ocr(plate_img)
        detected_chars = []

        for r in ocr_results:
            for box in r.boxes:
                x1c, y1c, x2c, y2c = map(int, box.xyxy[0])
                cls_id = int(box.cls[0])
                label = model_ocr.names[cls_id] if model_ocr.names and cls_id in model_ocr.names else ""
                x_center = (x1c + x2c) // 2
                detected_chars.append((x_center, label))

                abs_x1 = x1 + x1c
                abs_y1 = y1 + y1c
                abs_x2 = x1 + x2c
                abs_y2 = y1 + y2c

                cv2.rectangle(frame, (abs_x1, abs_y1), (abs_x2, abs_y2), (0, 255, 0), 1)
                cv2.putText(frame, label, (abs_x1, abs_y1 - 2),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)

        detected_chars.sort(key=lambda x: x[0])
        raw_plate = ''.join([char for _, char in detected_chars])

        if raw_plate:
            plate_history.append(raw_plate)

    if plate_history:
        most_common_raw, _ = Counter(plate_history).most_common(1)[0]
        final_plate = normalize_plate(most_common_raw)
        if final_plate:
            cv2.putText(frame, f"Detected: {final_plate}",
                        (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.8, (0, 255, 0), 2)

    cv2.imshow("ALPR Real-Time", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 127.4ms
Speed: 3.8ms preprocess, 127.4ms inference, 21.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 33.3ms
Speed: 4.4ms preprocess, 33.3ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 42.6ms
Speed: 2.2ms preprocess, 42.6ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 43.0ms
Speed: 6.2ms preprocess, 43.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 60.0ms
Speed: 1.5ms preprocess, 60.0ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 51.0ms
Speed: 3.6ms preprocess, 51.0ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 42.0ms
Speed: 1.7ms preprocess, 42.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 45.0ms
Speed: 2.0ms preprocess, 45.0m